In [1]:
import torch
import torch.nn.functional as F
import torchvision.models as models
import matplotlib.pyplot as plt
import sys
import getopt
import pathlib
import shutil
import numpy as np
from matplotlib.image import imsave
from glob import glob
from os.path import join
from utils import load_image, apply_transforms_score, basic_visualize, visualize, save_output
from cam.scorecam import ScoreCAM
import warnings

In [3]:
## EXPERIMENT 1: Class discrimination

# clear results directory
warnings.filterwarnings("ignore")
path_results = pathlib.Path('./results')
if path_results.exists() and path_results.is_dir():
    shutil.rmtree(path_results) #rm previous results
path_results.mkdir(parents=True, exist_ok=True) 

# init
TARGET_LAYER = "features.29"
IMAGE_DIR = "images/X_ILSVRC2012"
MEANS = [0.485, 0.456, 0.406]
STDS = [0.229, 0.224, 0.225]
SIZE = 224

# load model
model = models.vgg16(pretrained=True).eval()
model_scorecam = ScoreCAM(dict(arch=model, layer_name=TARGET_LAYER, input_size=(SIZE, SIZE)))

# load images
images = glob(join(IMAGE_DIR, "*"))

# preprocess image + generate heatmaps
for index, image in enumerate(images):
    
    # preprocess
    input_image = load_image(image)  
    input_ = apply_transforms_score(input_image, MEANS, STDS, SIZE)
    if torch.cuda.is_available():
        input_ = input_.cuda()

    # get class
    predicted_class = model(input_).max(1)[-1]
    
    # get heatmap
    heatmap = model_scorecam(input_)

    print(heatmap.shape)
    

# visualize heatmaps

Target layer:  ReLU(inplace=True)
torch.Size([1, 3, 224, 224])


/Users/duc/anaconda3/envs/scorecam/lib/python3.10/site-packages/torch/nn/modules/module.py:1033: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


torch.Size([1, 1, 224, 224])
